In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


In [2]:
traine_df = pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')
traine_df.head()
 


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [3]:
trains_df = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet")

In [4]:
! pip install datasets

In [5]:
from datasets import load_dataset


In [6]:
dataset = load_dataset("parquet", data_files="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-2cd1642c8462f6ce/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['series_id', 'step', 'timestamp', 'anglez', 'enmo'],
        num_rows: 127946340
    })
})

In [16]:
dataset['train']

Dataset({
    features: ['series_id', 'step', 'timestamp', 'anglez', 'enmo'],
    num_rows: 127946340
})

In [14]:
def transform_date(example):
    example['train']["timestamp"] = example['train']["timestamp"].split("-")[0] # Extract the year from the date
    return example

In [15]:
dataset.map(transform_date,batched=True)

  0%|          | 0/127947 [00:00<?, ?ba/s]

KeyError: 'train'

In [ ]:
exp_df['series_id'].nunique()


In [ ]:
"/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet"

In [ ]:
traine_df = traine_df.dropna()


In [ ]:
traine_df

In [ ]:
exp_df['Date'] = exp_df['timestamp'].str[:10]# Extract the first 10 characters for the Date

exp_df['Time'] = exp_df['timestamp'].str[11:19] 
exp_df['Timezone'] = exp_df['timestamp'].str[19:]

In [ ]:
exp_df.drop(columns=['timestamp'], inplace=True)

In [ ]:
exp_df.head()

In [ ]:
 traine_df.drop(columns=['timestamp'], inplace=True)

In [ ]:
merged_df = exp_df.merge(traine_df[['step','event']], on='step' , how='left')

In [ ]:
merged_df['event'].fillna('-', inplace=True)

In [ ]:
merged_df

In [ ]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

In [ ]:
merged_df['MonthNum'] = merged_df['Date'].dt.month
merged_df['Weekday'] = merged_df['Date'].dt.weekday
merged_df['year'] = merged_df['Date'].dt.year

In [ ]:
merged_df['day'] = merged_df['Date'].dt.day

reduce ram usage


In [ ]:

del traine_df, trains_df , exp_df

In [ ]:
merged_df['Hour'] = merged_df['Time'].str[:2]

In [ ]:
merged_df['min'] = merged_df['Time'].str[3:5]


In [ ]:
merged_df['sec'] = merged_df['Time'].str[6:]


In [ ]:
merged_df.drop(columns="Time" , inplace=True)

In [ ]:
merged_df

In [ ]:
merged_df.drop(columns="Date" , inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
merged_df['series_id'] = le.fit_transform(merged_df['series_id'])


In [ ]:
X = merged_df.drop('event', axis=1)  # Features
Y = merged_df['event']  # Labels

In [ ]:
del merged_df

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
del X , Y

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=.2)

In [ ]:
X_train.head()

In [ ]:
from xgboost import XGBClassifier

In [ ]:
clf = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=500, objective='binary:logistic', booster='gbtree', tree_method='hist')
clf.fit(X_train, y_train)


In [ ]:

# Make predictions and evaluate model
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score



In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
